In [6]:
!pip3 install requests folium seaborn rasterstats pystac_client pandas matplotlib

Defaulting to user installation because normal site-packages is not writeable
  Using cached seaborn-0.13.2-py3-none-any.whl.metadata (5.4 kB)
Using cached seaborn-0.13.2-py3-none-any.whl (294 kB)

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: /Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip


In [7]:
# Import the following libraries
import requests
import folium
import folium.plugins
from folium import Map, TileLayer
from pystac_client import Client
import branca
import pandas as pd
import matplotlib.pyplot as plt
import branca.colormap as cm
import seaborn as sns

In [8]:
# Provide the STAC and RASTER API endpoints
# The endpoint is referring to a location within the API that executes a request on a data collection nesting on the server.

# The STAC API is a catalog of all the existing data collections that are stored in the GHG Center.
STAC_API_URL = "https://earth.gov/ghgcenter/api/stac"

# The RASTER API is used to fetch collections for visualization
RASTER_API_URL = "https://earth.gov/ghgcenter/api/raster"

# The collection name is used to fetch the dataset from the STAC API. First, we define the collection name as a variable
# Name of the collection for methane emission plumes 
collection_name = "emit-ch4plume-v1"

In [9]:
# Fetch the collection from the STAC API using the appropriate endpoint
# The 'requests' library allows a HTTP request possible
collection = requests.get(f"{STAC_API_URL}/collections/{collection_name}").json()

# Print the properties of the collection to the console
collection

{'id': 'emit-ch4plume-v1',
 'type': 'Collection',
 'links': [{'rel': 'items',
   'type': 'application/geo+json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/emit-ch4plume-v1/items'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/'},
  {'rel': 'self',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/emit-ch4plume-v1'}],
 'title': 'EMIT Methane Point Source Plume Complexes v1',
 'extent': {'spatial': {'bbox': [[-121.90661810411778,
     -46.67515686526545,
     151.4637026107715,
     51.82354936938456]]},
  'temporal': {'interval': [['2022-08-10 06:49:57+00',
     '2024-09-02 13:08:32+00']]}},
 'license': 'CC-BY-1.0',
 'renders': {'dashboard': {'bidx': 1,
   'assets': ['ch4-plume-emissions'],
   'nodata': -9999,
   'rescale': [[1, 1500]],
   'resampling': 'bilinear',
   'col

In [10]:
def get_item_count(collection_id):
    count = 0
    items_url = f"{STAC_API_URL}/collections/{collection_id}/items"

    while True:
        response = requests.get(items_url)

        if not response.ok:
            print("error getting items")
            exit()

        stac = response.json()
        count += int(stac["context"].get("returned", 0))
        next = [link for link in stac["links"] if link["rel"] == "next"]

        if not next:
            break
        items_url = next[0]["href"]

    return count

In [11]:
# Check total number of items available
number_of_items = get_item_count(collection_name)
items = requests.get(f"{STAC_API_URL}/collections/{collection_name}/items?limit={number_of_items}").json()["features"]
print(f"Found {len(items)} items")

Found 1493 items


In [12]:
# The collection name is used to fetch the dataset from the STAC API. First, we define the collection name as a variable
# Name of the collection for methane emission plumes 
collection_name = "emit-ch4plume-v1"


# Fetch the collection from the STAC API using the appropriate endpoint
# The 'requests' library allows a HTTP request possible
collection = requests.get(f"{STAC_API_URL}/collections/{collection_name}").json()

# Print the properties of the collection in a table
# Adjust display settings
pd.set_option('display.max_colwidth', None)  # Set maximum column width to "None" to prevent cutting off text

# Extract the relevant information about the collection
collection_info = {
    "Title": collection.get("title", "N/A"), # Extract the title of the collection 
    "Description": collection.get("description", "N/A"), # Extract the dataset description
    "Temporal Extent": collection.get("extent", {}).get("temporal", {}).get("interval", "N/A"), # Extract the temporal coverage of the collection
    "Spatial Extent": collection.get("extent", {}).get("spatial", {}).get("bbox", "N/A"), # Extract the spatial coverage of the collection
}

# Convert the derived information into a DataFrame format
properties_table = pd.DataFrame(list(collection_info.items()), columns=["Collection Summary", ""])

# Display the properties in a table
collection_summary = properties_table.style.set_properties(**{'text-align': 'left'}) \
                                           .set_table_styles([
    {
        'selector': 'th.col0, td.col0',    # Select the first column
        'props': [('min-width', '200px'),  # Set a minimum width
                  ('text-align', 'left')]  # Align text to the left
    },
    {
        'selector': 'td.col1',             # Select the second column
        'props': [('text-align', 'left')]  # Align text to the left
    }
])

# Print the collection summary table
collection_summary

,Collection Summary,
0,Title,EMIT Methane Point Source Plume Complexes v1
1,Description,"This dataset provides methane plume complexes at 60 meter resolution measured with the EMIT instrument on the International Space Station (ISS) in units of parts per million meter (ppm-m). Due to its position on the ISS, EMIT does not sample everywhere on Earth nor can methane plumes be derived for all locations observed. By measuring the spectral fingerprint of methane, EMIT can map areas of high methane concentration over background levels in the atmosphere, identifying plume complexes, and estimating the methane enhancements. Large methane emissions, typically referred to as point source emissions, represent a significant proportion of total methane emissions from the production, transport, and processing of oil and natural gas, landfills, and other sources. Data is available starting August 1, 2022 and data collection is ongoing. The source data and additional information can be found at https://doi.org/10.5067/EMIT/EMITL2BCH4PLM.001"
2,Temporal Extent,"[['2022-08-10 06:49:57+00', '2024-09-02 13:08:32+00']]"
3,Spatial Extent,"[[-121.90661810411778, -46.67515686526545, 151.4637026107715, 51.82354936938456]]"


In [13]:
# Create a function that would search for data collection in the US GHG Center STAC API

# First, we need to define the function
# The name of the function is "get_item_count" 
# The argument that will be passed to the defined function is "collection_id"
def get_item_count(collection_id):

    # Set a counter for the number of items existing in the collection 
    count = 0 

    # Define the path to retrieve the granules (items) of the collection of interest in the STAC API
    items_url = f"{STAC_API_URL}/collections/{collection_id}/items" 

    # Run a while loop to make HTTP requests until there are no more URLs associated with the collection in the STAC API
    while True:

        # Retrieve information about the granules by sending a "get" request to the STAC API using the defined collection path 
        response = requests.get(items_url) 

        # If the items do not exist, print an error message and quit the loop
        if not response.ok:
            print("error getting items")
            exit()

        # Return the results of the HTTP response as JSON
        stac = response.json()

        # Increase the "count" by the number of items (granules) returned in the response
        count += int(stac["context"].get("returned", 0))

        # Retrieve information about the next URL associated with the collection in the STAC API (if applicable)
        next = [link for link in stac["links"] if link["rel"] == "next"]

        # Exit the loop if there are no other URLs
        if not next:
            break
        
        # Ensure the information gathered by other STAC API links associated with the collection are added to the original path
        # "href" is the identifier for each of the tiles stored in the STAC API
        items_url = next[0]["href"]

    # Return the information about the total number of granules found associated with the collection
    return count

In [15]:
# Import the following libraries
import requests
import folium
import folium.plugins
from folium import Map, TileLayer 
from pystac_client import Client 
import branca 
import pandas as pd
import matplotlib.pyplot as plt
from tabulate import tabulate
import branca.colormap as cm
import seaborn as sns

In [16]:
# Apply the function created above "get_item_count" to the collection
number_of_items = get_item_count(collection_name)

# Get the information about the number of granules found in the collection
items = requests.get(f"{STAC_API_URL}/collections/{collection_name}/items?limit={number_of_items}").json()["features"]

# Print the total number of items (granules) found
print(f"Found {len(items)} observations")

# Sort the items based on their date-time attribute
items_sorted = sorted(items, key=lambda x: x["properties"]["datetime"])

# Create an empty list
table_data = []
# Extract the ID and date-time information for each granule and add them to the list
# By default, only the first 5 items in the collection are extracted to be displayed in the table. 
# To see the date-time of all existing granules in this collection, remove "5" from "item_sorted[:5]" in the line below. 
for item in items_sorted[:5]:
    table_data.append([item['id'], item['properties']['datetime']])

# Define the table headers
headers = ["Item ID", "Date-Time"]

print("Below you see the first 5 items in the collection, along with their item IDs and corresponding Start Date-Time.")

# Print the table using tabulate
print(tabulate(table_data, headers=headers, tablefmt="fancy_grid"))

Found 1493 observations
Below you see the first 5 items in the collection, along with their item IDs and corresponding Start Date-Time.
╒════════════════════════════════════════════╤═══════════════════════════╕
│ Item ID                                    │ Date-Time                 │
╞════════════════════════════════════════════╪═══════════════════════════╡
│ EMIT_L2B_CH4PLM_001_20220810T064957_000485 │ 2022-08-10T06:49:57+00:00 │
├────────────────────────────────────────────┼───────────────────────────┤
│ EMIT_L2B_CH4PLM_001_20220810T065021_000486 │ 2022-08-10T06:50:21+00:00 │
├────────────────────────────────────────────┼───────────────────────────┤
│ EMIT_L2B_CH4PLM_001_20220810T065132_000496 │ 2022-08-10T06:51:32+00:00 │
├────────────────────────────────────────────┼───────────────────────────┤
│ EMIT_L2B_CH4PLM_001_20220810T065132_000487 │ 2022-08-10T06:51:32+00:00 │
├────────────────────────────────────────────┼───────────────────────────┤
│ EMIT_L2B_CH4PLM_001_20220811T042630_0

In [17]:
# Examine the first item in the collection
# Keep in mind that a list starts from 0, 1, 2... therefore items[0] refers to the first item (granule) in the list/collection
items_sorted[0]

{'id': 'EMIT_L2B_CH4PLM_001_20220810T064957_000485',
 'bbox': [35.192241059678175,
  31.111893793152127,
  35.22206384829227,
  31.13575202404341],
 'type': 'Feature',
 'links': [{'rel': 'collection',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/emit-ch4plume-v1'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/emit-ch4plume-v1'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/'},
  {'rel': 'self',
   'type': 'application/geo+json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/emit-ch4plume-v1/items/EMIT_L2B_CH4PLM_001_20220810T064957_000485'},
  {'title': 'Map of Item',
   'href': 'https://earth.gov/ghgcenter/api/raster/collections/emit-ch4plume-v1/items/EMIT_L2B_CH4PLM_001_20220810T064957_000485/map?bidx=1&assets=ch4-plume-emissions&nodata=-9999&rescale=1%2C1500&resampling=bilinear&colormap_name=plasma',
   

In [18]:
# Once again, apply the function created above "get_item_count" to the Air-Sea CO2 Flux ECCO-Darwin collection
# This step allows retrieving the number of granules “observations” in the collection.
number_of_items = get_item_count(collection_name)
items = requests.get(f"{STAC_API_URL}/collections/{collection_name}/items?limit={number_of_items}").json()["features"]


# Next, you need to create a dictionary where the "id" field of each item in the collection are queried more explicitly
plume_complexes = {items["id"]: items for items in items} 


# Next, you need to specify the asset name for this collection.
# The asset name refers to the raster band containing the pixel values for the parameter of interest.
# For the case of the EMIT Methane Point Source collection, the parameter of interest is “ch4-plume-emissions”.
asset_name = "ch4-plume-emissions"

In [19]:
# Fetching the min and max values for a specific item
rescale_values = {"max":plume_complexes[list(plume_complexes.keys())[0]]["assets"][asset_name]["raster:bands"][0]["histogram"]["max"], "min":plume_complexes[list(plume_complexes.keys())[0]]["assets"][asset_name]["raster:bands"][0]["histogram"]["min"]}

In [20]:
# Select the item ID which you want to visualize. Item ID is in the format yyyymmdd followed by the timestamp. This ID can be extracted from the COG name as well.
# To browse and select other tiles in the collection, please visit https://search.earthdata.nasa.gov/search/granules?p=C2748088093-LPCLOUD&pg[0][v]=f&pg[0][gsk]=-start_date&q=emit%20plume&tl=1694622854.77!3!!

# You need to copy the entire granule nomenclature 
item_id = "EMIT_L2B_CH4PLM_001_20230418T200118_000829"

# Choose a color map for displaying the first observation (event)
# Please refer to matplotlib library if you'd prefer to choose a different color ramp.
# For more information on Colormaps in Matplotlib, please visit https://matplotlib.org/stable/users/explain/colors/colormaps.html
color_map = "magma"

# Make a GET request to retrieve information for the selected tile defined in "item_id"
methane_plume_tile = requests.get(
    f"{RASTER_API_URL}/collections/{plume_complexes[item_id]['collection']}/items/{plume_complexes[item_id]['id']}/tilejson.json?"
    f"&assets={asset_name}"
    
    # Pass the color formula and colormap for custom visualization
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
    
    # Pass the minimum and maximum values for rescaling 
    f"&rescale={rescale_values['min']},{rescale_values['max']}", 
    
# Return the response in JSON format
).json()

# Print the properties of the retrieved granule to the console
methane_plume_tile

{'tilejson': '2.2.0',
 'version': '1.0.0',
 'scheme': 'xyz',
 'tiles': ['https://earth.gov/ghgcenter/api/raster/collections/emit-ch4plume-v1/items/EMIT_L2B_CH4PLM_001_20230418T200118_000829/tiles/WebMercatorQuad/{z}/{x}/{y}@1x?assets=ch4-plume-emissions&color_formula=gamma+r+1.05&colormap_name=magma&rescale=-1546.5286865234375%2C5230.7080078125'],
 'minzoom': 0,
 'maxzoom': 24,
 'bounds': [-104.76285251117253,
  39.85322425220504,
  -104.74658553556483,
  39.86515336765068],
 'center': [-104.75471902336868, 39.85918880992786, 0]}

In [45]:
import folium
import requests
from branca.colormap import LinearColormap

# カラーマップを定義
colormap = LinearColormap(
    colors=['#310597', '#4C02A1', '#6600A7', '#7E03A8', '#9511A1', '#AA2395', '#BC3587', '#CC4778', '#DA5A6A', '#E66C5C', '#F0804E', '#F89540','#FDAC33', '#FDC527', '#F8DF25'],
    vmin=0, vmax=1500
)
colormap.caption = 'ppm-m'


# 地図の初期設定
map_ = folium.Map(location=[methane_plume_tile["center"][1], methane_plume_tile["center"][0]], zoom_start=14)

color = colormap((rescale_values['max'] - rescale_values['min']) / 2)

folium.Marker(
    location=[methane_plume_tile["center"][1], methane_plume_tile["center"][0]],
    popup=item_id,
    icon=folium.Icon(color='red', icon='info-sign', icon_color=color)
).add_to(map_)


# カラーマップを追加
map_.add_child(colormap)

# 地図を表示
map_

In [22]:
# Access the coordinates of the plume feature (using the 'item_id')
plumes_coordinates = plume_complexes[item_id]["geometry"]["coordinates"]

# Create a polygon for the area of interest (aoi)
methane_plume_aoi = {
    "type": "Feature", # Create a feature object
    "properties": {},
    "geometry": {      # The geometry of the feature
        "coordinates":
            plumes_coordinates,  # Use the plume's coordinates retrieved earlier
        "type": "Polygon",
    },
}

In [23]:
# Please put the name of the place you are trying to visualize. 
# The granule that was selected by default is showuing plumes near Denver, United States. 
region_name = "Place_Holder"


# Create a new map to display the generated polygon
aoi_map = Map(
    
    # Base map is set to OpenStreetMap
    tiles="OpenStreetMap",
    
    # Define the spatial properties for the map
    location=[
        plumes_coordinates[0][0][1],
        plumes_coordinates[0][0][0]
    ],
    
    # Set the center of the map
    zoom_start=12,
)

# Insert the polygon to the map
folium.GeoJson(methane_plume_aoi, name=region_name).add_to(aoi_map)


# Visualize the map
aoi_map

In [24]:
# Check the total number of items available within the collection
items = requests.get(
    f"{STAC_API_URL}/collections/{collection_name}/items?limit={number_of_items}"
).json()["features"]

# Print the total number of items (granules) found
print(f"Found {len(items)} items")

Found 1493 items


In [25]:
# Examine the first item in the collection
# Keep in mind that a list starts from 0, 1, 2... therefore items[0] refers to the first item (granule) in the list/collection
items[0]

{'id': 'EMIT_L2B_CH4PLM_001_20240902T130832_003513',
 'bbox': [-47.59853129566521,
  -23.467062765772685,
  -47.52153427778881,
  -23.41392397878756],
 'type': 'Feature',
 'links': [{'rel': 'collection',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/emit-ch4plume-v1'},
  {'rel': 'parent',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/emit-ch4plume-v1'},
  {'rel': 'root',
   'type': 'application/json',
   'href': 'https://earth.gov/ghgcenter/api/stac/'},
  {'rel': 'self',
   'type': 'application/geo+json',
   'href': 'https://earth.gov/ghgcenter/api/stac/collections/emit-ch4plume-v1/items/EMIT_L2B_CH4PLM_001_20240902T130832_003513'},
  {'title': 'Map of Item',
   'href': 'https://earth.gov/ghgcenter/api/raster/collections/emit-ch4plume-v1/items/EMIT_L2B_CH4PLM_001_20240902T130832_003513/map?bidx=1&assets=ch4-plume-emissions&nodata=-9999&rescale=1%2C1500&resampling=bilinear&colormap_name=plasma',


In [26]:
# The bounding box should be passed to the geojson param as a geojson Feature or FeatureCollection
# Create a function that retrieves information regarding a specific granule using its asset name and raster identifier and generates the statistics for it

# The function takes an item (granule) and a JSON (polygon) as input parameters
def generate_stats(item, geojson):
    
    # A POST request is made to submit the data associated with the item of interest (specific observation) within the boundaries of the polygon to compute its statistics
    result = requests.post(
        
        # Raster API Endpoint for computing statistics
        f"{RASTER_API_URL}/cog/statistics",
        
        # Pass the URL to the item, asset name, and raster identifier as parameters
        params={"url": item["assets"][asset_name]["href"]},
        
        # Send the GeoJSON object (polygon) along with the request
        json=geojson,
        
    # Return the response in JSON format
    ).json()
    
    # Print the result
    print(result)
    
    # Return a dictionary containing the computed statistics along with the item's datetime information.
    return {
        **result["properties"],
        "item_id": item["id"][20:],
    }

In [27]:
# Generate a for loop that iterates over all the existing items in the collection 
for item in items:
    
    # The loop will then retrieve the information for the start datetime of each item in the list
    #print(item["id"])
    print(item["properties"]["datetime"])
    
    # Exit the loop after printing the start datetime for the first item in the collection
    break

2024-09-02T13:08:32+00:00


In [29]:
%%time
# %%time = Wall time (execution time) for running the code below

# Generate statistics using the created function "generate_stats" within the bounding box defined by the aoi polygon
stats = [generate_stats(item, methane_plume_aoi) for item in items]
stats = [ stat for stat in stats if stat["statistics"]["b1"]["mean"] != None]

{'detail': 'Internal Server Error'}


KeyError: 'properties'

In [30]:
def clean_stats(stats_json) -> pd.DataFrame:
    df = pd.json_normalize(stats_json)
    df.columns = [col.replace("statistics.b1.", "") for col in df.columns]
    # df["date"] = pd.to_datetime(df["datetime"])
    return df


df = clean_stats(stats)
df

NameError: name 'stats' is not defined

In [ ]:
plume_tile_2 = requests.get(
    f"{RASTER_API_URL}/collections/{items[0]['collection']}/items/{items[0]['id']}/tilejson.json?"
    f"&assets={asset_name}"
    f"&color_formula=gamma+r+1.05&colormap_name={color_map}"
    f"&rescale={rescale_values['min']},{rescale_values['max']}",
).json()
plume_tile_2

In [ ]:
# Use bbox initial zoom and map
# Set up a map located w/in event bounds
plume_tile_2_coordinates = items[0]["geometry"]["coordinates"]
aoi_map_bbox = Map(
    tiles="OpenStreetMap",
    location=[
        plume_tile_2_coordinates[0][0][1],
        plume_tile_2_coordinates[0][0][0]
    ],
    zoom_start=10,
)

map_layer = TileLayer(
    tiles=plume_tile_2["tiles"][0],
    attr="GHG", opacity = 1
)

map_layer.add_to(aoi_map_bbox)

aoi_map_bbox